In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
import sys
sys.path.append('/content/gdrive/MyDrive/Colab Notebooks')
import my_utils as mu
import torch
import collections
from collections import defaultdict
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib
import matplotlib.pyplot as plt
from IPython import display
import numpy as np
import pandas as pd
import random
import math
from torch.utils.data.sampler import SubsetRandomSampler
d2l = sys.modules[__name__]
## Code to train a specific MLP model in recognising Fashion-MNIST datasets.


## First, the dataset is retrieved, then transformed to fit our split algorithm.
shift = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.1307,), (0.3081,))])
batch_size = 256

fashion_trainset = datasets.FashionMNIST(root='./fmnist/', train=True,
                                         download=True, transform=shift)
train_range = len(fashion_trainset)
indexes = list(range(train_range))
divide = 24000

## The data is split randomly in to test and train datasets. The greater our 
## split is in favour of train, the more accurate our test will be.
val = np.random.choice(indexes, size=divide, replace=True)
train = list(set(indexes) - set(val))

sampler1 = SubsetRandomSampler(train)
sampler2 = SubsetRandomSampler(val)

## Finally, dataloaders are initialised for later use in the trainer model.
train_iter = torch.utils.data.DataLoader(fashion_trainset, batch_size, 
                                         sampler=sampler1)
test_iter = torch.utils.data.DataLoader(fashion_trainset, batch_size,
                                         sampler=sampler2)
train_features, train_labels = next(iter(train_iter))



num_inputs = 49
num_hidden = 256
num_outputs = 10

## Next, we have our stem/backbone/classifier model
class mlp(nn.Module):
        def __init__(self, num_inputs, num_hidden, num_outputs):
            super(mlp, self).__init__()
            
            ## Initialise our inputs, and the patch variables
            self.num_inputs = num_inputs
            self.num_hidden = num_hidden
            self.num_outputs = num_outputs
            self.number = 16
            self.dimensions = 7
            self.pixels = 49

            ## Linear function and 2 sequentially run backbones, with the 
            ## tranposed inputs added later.
            self.side = nn.Linear(self.pixels, num_inputs)

            self.backbone1 = nn.Sequential(nn.Linear(num_inputs, num_hidden, True), 
                                      nn.ReLU(), nn.Linear(num_hidden, num_hidden, True))
            self.backbone2 = nn.Sequential(nn.Linear(num_hidden, num_hidden, True), 
                                      nn.ReLU(), nn.Linear(num_hidden, num_outputs, True))


        def forward(self, x):
            
           ## Stem process to arrange oncoming data from tensors in to small 
           ## pixels flattened to be side by side.
           unfold = nn.Unfold(kernel_size=(7,7), stride=(7,7))
           x = x.unfold(2, 7, 7).unfold(3, 7, 7)
           tens = []
           for patch in x:
              tens2 = []
              p = patch.flatten()
              for patch2 in torch.split(p, self.pixels):
                 final_patch = self.side(patch2)
                 tens2.append(final_patch)
              tens.append(torch.stack(tens2))

           ## Data is then individually transposed and put through each backbone
           x = torch.stack(tens)
           x = torch.transpose(x, 0, 1)
           x = self.backbone1(x)
           x = torch.transpose(x, 0, 1)
           x = self.backbone2(x)
           
           ## Data is finally run through the classifier
           x = torch.mean(x, 1, False)
           return x
## Here, the weights of the model are initialised.
def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.normal_(m.weight, std=0.01)
        torch.nn.init.zeros_(m.bias)

## More variable initialising ahead of running the training model, including the
## loss, epochs and Adam optimiser.
net = mlp(num_inputs, num_hidden, num_outputs)
loss = nn.CrossEntropyLoss()
optimiser = optim.Adam(net.parameters(), lr=0.001, betas=(0.9, 0.999), 
                       eps=1e-08, weight_decay=0.001)
num_epochs = 25

## Finally, we use the train model included in the my_utils file. The train_iter 
## is provided to the model, which then analyses the test_iter data. As more 
## train_iter data is provided, the model becomes more accurate.
mu.train_ch3(net, train_iter, test_iter, loss, num_epochs, optimiser)

## Final accuracy is given above our graph animation from the above model.
mu.evaluate_accuracy(net, test_iter)
